In [50]:
##For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

In [51]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Install website scraping libraries and packages in Python from BeautifulSoup 
#!conda install -c conda-forge beautifulsoup4 --yes  # uncomment this line if you haven't completed 
from bs4 import BeautifulSoup as bs

print('Libraries imported.')



Libraries imported.


In [52]:
##1. Download Dataset
##From the Wikipage given the List of postal codes of Canada :
##Download the HTML file at the given link : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
##Register the file locally
##Open the file and iterate through HTML elements to extract postal codes using 'BeautifulSoup' library
##Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and longitude coordinates of each neighborhood.
##Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
##I downloaded the file from wikipedia web site and placed it locally using simply a wget command and access the data. So let's go ahead and do that.

!wget -q -O 'canada_postal_code_list_from_wikipedia.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('HTML Postal Code page downloaded!')

HTML Postal Code page downloaded!


In [53]:
##Load the data

# Get HTML content
with open("canada_postal_code_list_from_wikipedia.html") as fp:
    soup = bs(fp, 'lxml')

# Get the HTML table codes
tagTable = soup.table
#Get table body
body = tagTable.tbody



In [54]:
##Tranform the data into a pandas dataframe
##The task is essentially transforming this HTML data into a pandas dataframe. 
##So let's start by creating an empty dataframe with just the column names

# Define the dataframe columns 
# get table column names -> all 'th' tags of the body in 'tr' fields
colTab = (body.tr).find_all('th')
#print (colTab)
colNames = [(bs(str(colTab[i]))).find('th').string.strip() for i in range (0,3)]

# instantiate the dataframe
postcode_df = pd.DataFrame(columns=colNames)
postcode_df

,Postal Code,Borough,Neighbourhood


In [55]:
##Then let's loop through the data and fill the dataframe one row at a time.


postcode_df = pd.DataFrame(columns=colNames)

# extract all 'tr' tagged fields except the first one (column names)
codesTab= body.find_all('tr')[1:]

for n, code in enumerate(codesTab):
    # n.th postal code either : name or link
    #print ("\n", n ,".th",  code, )
    # for each element code 
    tabc = ["","",""]
    for i, value in enumerate(code.stripped_strings):
        tabc[i] = value.strip()
    #print("tabc", tabc)
    # Ignore cells with a borough that is Not assigned.
    #print(tabc[1], 'Not assigned', tabc[1] == 'Not assigned')
    postcode = tabc[0]
    borough = tabc[1]
    neighborhood = tabc[2]
    
    if borough != 'Not assigned':
        # insert
        # check a neighborhood is assigned else set it with borough
        if neighborhood == 'Not assigned':
            neighborhood = borough
        # insert the built postal code into the dataframe
        postcode_df = postcode_df.append({'Postcode' : postcode,
                            'Borough' : borough,
                            'Neighborhood': neighborhood},
                           ignore_index=True)

# Combine rows with same postal code into one row with the neighborhoods separated with a comma 
df = postcode_df.groupby('Postcode', as_index=False).agg({'Borough':'first', 'Neighborhood':', '.join})

##end of 1st part of this projet .shape
print ("Toronto postal codes dataframe dimensions = ", df.shape)
df.head(15)

Toronto postal codes dataframe dimensions =  (103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [56]:
###############################end of 1st part

In [57]:
##############################begin of 2nd part

In [58]:
# Store dataframe locally as a csv file to be easily used later
df.to_csv('canada_postal_code_list.csv')

In [59]:
##Add Geographical Coordinates
##Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
##in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

##Let's use the csv file.


In [60]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data   
print('Postal Code Coordinates downloaded!')

df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

df_geo_coor = pd.read_csv('Geospatial_Coordinates.csv')
df_geo_coor.head()


# Rename the Postal Code column to allow merging
df_coordinates.rename(columns={'Postal Code':'Postcode'}, inplace=True)
print(df_coordinates[0:3])

# for each postal code get the latitude and longitude values
# Merge the 2 dtaframe on the Postalcode column
df.head(10)
df_latlg = pd.merge(df, df_coordinates, on='Postcode' )
df_latlg.head(15)

Postal Code Coordinates downloaded!
  Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [61]:
#########################end of 2nd part

In [62]:
#########################begin of 3rd part Explore and cluster the neighborhoods in Toronto. 

In [63]:
#Explore and cluster the neighborhoods in Toronto
#Get the latitude and longitude values of Toronto.

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [64]:
#Create a map of the whole Toronto City with neighborhoods superimposed on top

In [65]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [66]:
for lat, lng, borough, neighborhood in zip(
        df_latlg['Latitude'], 
        df_latlg['Longitude'], 
        df_latlg['Borough'], 
        df_latlg['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [70]:
#Map of a part of Toronto City
#We are going to work with only the boroughs that contain the word "Toronto".

#"denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_latlg_denc = df_latlg[df_latlg['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_latlg_denc.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [ ]:
#Plot again the map and the markers for this region.

In [71]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_latlg_denc['Latitude'], 
        df_latlg_denc['Longitude'], 
        df_latlg_denc['Borough'], 
        df_latlg_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc

In [ ]:
#Define Foursquare Credentials and Version

In [73]:
CLIENT_ID = 'JUJKJFJJFXP1LTUGGN55JQIAPMEMKMVGKOIZMILBKYDLPS42'
CLIENT_SECRET = 'TPYHEDWXI43VHCAAGOZPUL1QK0SPDGHV221AMQI1WUSBZRMD'
VERSION = '20180604'

In [ ]:
#Explore the first neighborhood in our data frame "df_latlg"

In [74]:
neighborhood_name = df_latlg_denc.loc[0, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'The Beaches'.


In [75]:
#Get the neighborhood's latitude and longitude values.

In [76]:
neighborhood_latitude = df_latlg_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_latlg_denc.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [77]:
#Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [78]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [79]:
#Function that extracts the category of the venue

In [80]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [81]:
#Now we are ready to clean the json and structure it into a pandas dataframe.

In [82]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


In [83]:
#Explore neighborhoods in a part of Toronto City

In [ ]:
#We are working on the data frame df_latlg_denc. Recall that, this region contain DENC of Toronto where,
#"DENC" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
#First, let's create a function to repeat the same process to all the neighborhoods in DENC of Toronto.

In [127]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [129]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues

latlg_denc_venues = getNearbyVenues(names=df_latlg_denc['Neighborhood'],
                                   latitudes=df_latlg_denc['Latitude'],
                                   longitudes=df_latlg_denc['Longitude']
                                  )

In [130]:
latlg_denc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [131]:
#Let's check how many venues were returned for each neighborhood.

latlg_denc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [132]:
#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(latlg_denc_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [134]:
#Analyze Each Neighborhood
# one hot encoding
latlg_denc_onehot = pd.get_dummies(latlg_denc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
latlg_denc_onehot['Neighborhood'] = latlg_denc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [latlg_denc_onehot.columns[-1]] + list(latlg_denc_onehot.columns[:-1])
latlg_denc_onehot = latlg_denc_onehot[fixed_columns]

latlg_denc_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [135]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

latlg_denc_grouped = latlg_denc_onehot.groupby('Neighborhood').mean().reset_index()
latlg_denc_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.051724,0.000000,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.000000,0.017241,0.0000,0.0,0.0000,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.017241,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.017241,0.051724,0.086207,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017241,0.0,0.0,0.0,0.0,0.0

In [ ]:
#Check the 10 most common venues in each neighborhood.

In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_denc_grouped['Neighborhood']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_denc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Restaurant,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Restaurant,Stadium,Italian Restaurant,Bar,Bakery
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Park,Comic Shop,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Airport Service,Boat or Ferry,Boutique,Rental Car Location,Bar,Coffee Shop,Sculpture Garden,Airport Food Court
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Salad Place,Thai Restaurant,Ramen Restaurant,Restaurant


In [ ]:
#Cluster neighborhoods
#Run k-means to cluster the neighborhood into 5 clusters.

In [137]:
# set number of clusters
kclusters = 5

latlg_denc_grouped_clustering = latlg_denc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(latlg_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [138]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

latlg_denc_merged = df_latlg_denc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
latlg_denc_merged = latlg_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

latlg_denc_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Trail,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Yoga Studio,Spa,Bakery,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Ice Cream Shop,Steakhouse,Restaurant,Sushi Restaurant,Liquor Store,Italian Restaurant,Pub,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [ ]:
#Finally, let's visualize the resulting clusters

In [139]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        latlg_denc_merged['Latitude'], 
        latlg_denc_merged['Longitude'], 
        latlg_denc_merged['Neighborhood'], 
        latlg_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [140]:
#Examine Clusters
#Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#Cluster 1

latlg_denc_merged.loc[latlg_denc_merged['Cluster Labels'] == 0, latlg_denc_merged.columns[[1] + list(range(5, latlg_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Asian Restaurant,Health Food Store,Trail,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Yoga Studio,Spa,Bakery,Indian Restaurant
2,East Toronto,0,Park,Fast Food Restaurant,Ice Cream Shop,Steakhouse,Restaurant,Sushi Restaurant,Liquor Store,Italian Restaurant,Pub,Movie Theater
3,East Toronto,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant
5,Central Toronto,0,Gym / Fitness Center,Hotel,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gay Bar,Dance Studio
6,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Café,Seafood Restaurant,Spa,Sporting Goods Shop,Fast Food Restaurant,Salon / Barbershop,Diner
7,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Seafood Restaurant,Brewery
9,Central Toronto,0,Coffee Shop,Bagel Shop,Vietnamese Restaurant,Liquor Store,Bank,Sandwich Place,Supermarket,Health & Beauty Service,Pub,American Restaurant
11,Downtown Toronto,0,Restaurant,Café,Coffee Shop,Bakery,Italian Restaurant,Pub,Market,Pizza Place,Yoga Studio,Pharmacy
12,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Burger Joint,Pub,Hotel,Pizza Place


In [141]:
#Cluster 2

latlg_denc_merged.loc[latlg_denc_merged['Cluster Labels'] == 1, latlg_denc_merged.columns[[1] + list(range(5, latlg_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,1,Park,Playground,Trail,Women's Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run


In [142]:
#Cluster 3

latlg_denc_merged.loc[latlg_denc_merged['Cluster Labels'] == 2, latlg_denc_merged.columns[[1] + list(range(5, latlg_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Home Service,Garden,Women's Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [143]:
#Cluster 4

latlg_denc_merged.loc[latlg_denc_merged['Cluster Labels'] == 3, latlg_denc_merged.columns[[1] + list(range(5, latlg_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Park,Trail,Jewelry Store,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [144]:
#Cluster 5

latlg_denc_merged.loc[latlg_denc_merged['Cluster Labels'] == 4, latlg_denc_merged.columns[[1] + list(range(5, latlg_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Playground,Trail,Tennis Court,Women's Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop


In [ ]:
########################end of 3rd part

In [ ]:
#######################end of this final project assignment